In [1]:
with open('Day20.txt') as f:
    inputs = f.read().split('\n\n')

In [2]:
import numpy as np

def get_combinations(tile):
    for n in range(4):
        yield np.rot90(tile, k=n)
    for n in range(4):
        yield np.rot90(np.fliplr(tile), k=n)

class Tile:
    def __init__(self, number, origin):
        self.number = number
        self.origin = origin
        self.choice = None
        self.combinations = list(get_combinations(origin))
        self.borders = {
            tuple(combination[0,:].tolist()[0]): 0
            for combination in self.combinations}
    
    def __repr__(self):
        if self.choice is not None:
            return f"<Tile #{self.number}:{self.choice}>"
        return f"<Tile #{self.number}>"
    
    def __hash__(self):
        return self.number
    
    def check(self, top=None, lft=None):
        for index, combination in enumerate(self.combinations):
            if not top and not lft:
                col = tuple(combination[:,0].flatten().tolist()[0])
                row = tuple(combination[0,:].flatten().tolist()[0])
                if self.borders[col] == self.borders[row] == 0:
                    self.choice = index
                    return self
            else:
                vtop = (not top or (top.matrix[-1,:] == combination[0,:]).all())
                vlft = (not lft or (lft.matrix[:,-1] == combination[:,0]).all())
                if vtop and vlft:
                    self.choice = index
                    return self
    
    @property
    def matrix(self):
        if self.choice:
            return self.combinations[self.choice]
        return self.origin
    
    @classmethod
    def parse(cls, raw):
        lines = raw.replace('#', '1').replace('.', '0').split('\n')
        number = int(lines.pop(0).split()[1][:-1])
        origin = np.matrix(';'.join(','.join(line) for line in lines))
        return cls(number, origin)
    
    @staticmethod
    def count_borders(tiles):
        for tile in tiles.values():
            for other in tiles.values():
                if tile == other:
                    continue
                for border in other.borders:
                    if border not in tile.borders:
                        continue
                    tile.borders[border] += 1
        return {
            tile.number: sum(tile.borders.values()) 
            for tile in tiles.values()}

In [3]:
tiles = {}
for block in inputs:
    tile = Tile.parse(block)
    tiles[tile.number] = tile

In [4]:
%%time
from functools import reduce
from operator import mul, itemgetter

borders = sorted(Tile.count_borders(tiles).items(), key=lambda e: e[1])[:4]
reduce(mul, map(itemgetter(0), borders))

CPU times: user 37.7 ms, sys: 891 µs, total: 38.6 ms
Wall time: 37.8 ms


64802175715999

In [5]:
%%time
import math
puzzle = {}
pieces = set(tiles.keys())
size = int(math.sqrt(len(tiles)))
for row in range(size):
    for col in range(size):
        top_tile = puzzle.get((row - 1, col))
        lft_tile = puzzle.get((row, col - 1))
        for piece in pieces:
            tile = tiles[piece]
            if tile.check(top=top_tile, lft=lft_tile):
                puzzle[row, col] = tile
                pieces.remove(tile.number)
                break
full = np.bmat([[puzzle[col, row].matrix[1:-1,1:-1] for row in range(size)] for col in range(size)])

CPU times: user 1.44 s, sys: 0 ns, total: 1.44 s
Wall time: 1.44 s


In [6]:
monster = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """
monster = np.matrix(monster.replace(' ', '9,').replace('#', '1,').replace(',\n', ';'))
monster_count = np.count_nonzero(monster < 9)

In [7]:
def find_monsters(full):
    frows, fcols = full.shape
    mrows, mcols = monster.shape
    monsters = 0
    for row in range(frows - mrows):
        for col in range(fcols - mcols):
            tile = full[row:row+mrows,col:col+mcols]
            if np.count_nonzero(tile == monster) == monster_count:
                monsters += 1
                np.place(tile, monster < 9, 2)
    return monsters, np.count_nonzero(full == 1)

In [8]:
%%time
for combination in get_combinations(full):
    monsters, safes = find_monsters(combination)
    if monsters:
        break
monsters, safes

CPU times: user 218 ms, sys: 265 µs, total: 218 ms
Wall time: 217 ms


(35, 2146)